In [1]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
from __init__ import *
from __init__jupyterlab import *

import snmcseq_utils

In [18]:
df_genes = pd.read_csv(PATH_GENEBODY_ANNOTATION, sep='\t')
df_genes['gid'] = df_genes['gene_id'].apply(lambda x: x.split('.')[0])
_x = df_genes.set_index('gid')[['start', 'end']]
gene_lengths = _x['end'] - _x['start']
print(df_genes.shape)
df_genes.head()

(53379, 8)


,gene_id,gene_name,chr,start,end,strand,gene_type,gid
0,ENSMUSG00000102693.1,4933401J01Rik,chr1,3073253,3074322,+,TEC,ENSMUSG00000102693
1,ENSMUSG00000064842.1,Gm26206,chr1,3102016,3102125,+,snRNA,ENSMUSG00000064842
2,ENSMUSG00000051951.5,Xkr4,chr1,3205901,3671498,-,protein_coding,ENSMUSG00000051951
3,ENSMUSG00000102851.1,Gm18956,chr1,3252757,3253236,+,processed_pseudogene,ENSMUSG00000102851
4,ENSMUSG00000103377.1,Gm37180,chr1,3365731,3368549,-,TEC,ENSMUSG00000103377


In [3]:
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_clusterings.tsv'
df_clst = pd.read_csv(f, sep='\t')
print(df_clst.shape)
df_clst.head()

/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(408885, 11)


,sample,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4
0,snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1
1,snmcseq_gene_2C_M_1,-2.414973,-12.098081,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-2,4-2-1,4-2-1-1
2,snmcseq_gene_2C_M_100,-1.446751,-12.051894,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-1,4-1-1,4-1-1-1
3,snmcseq_gene_2C_M_1000,-1.901734,-13.660591,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-2,4-2-1,4-2-1-1
4,snmcseq_gene_2C_M_1001,-0.981086,-11.936621,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-1,4-1-1,4-1-1-1


In [13]:
mods_updated = [
    'smarter_cells',
    '10x_cells_v3',
    '10x_cells_v2', 
    'smarter_nuclei',
    '10x_nuclei_v3_macosko',
    '10x_nuclei_v3',
    'snatac_gene',
#     'mch',
#     'mcg',
]

rounds = ['round2', 'round3']

In [11]:
for mod in mods_updated:
    print(mod)
    
    f = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/{}_raw.npz'.format(mod)
    fgene = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/{}_raw.gene'.format(mod)
    fcell = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/{}_raw.cell'.format(mod)
    gxc_mat = snmcseq_utils.load_gc_matrix(fgene, fcell, f)

    df2 = df_clst[df_clst['modality']==mod]
    for _round in rounds:
        print(_round)
        cluster_level = 'joint_cluster_{}'.format(_round)
        clusters = np.sort(df_clst[cluster_level].unique())
        df_sum = pd.DataFrame(index=gxc_mat.gene, columns=clusters)
        for clst, df_sub in df2.groupby(cluster_level):
            print('.', end='')
            cells = df_sub['sample'].values
            cells_idx = snmcseq_utils.get_index_from_array(gxc_mat.cell, cells)
            df_sum[clst] = np.ravel(gxc_mat.data.tocsc()[:, cells_idx].sum(axis=1))
        f = '/cndd2/fangming/miniatlas/cluster_level/gene_counts_{}_{}.tsv'.format(mod, _round)
        df_sum.to_csv(f, na_rep='NA', sep="\t", index=True, header=True)

smarter_cells
round2
.............................round3
......................................................10x_cells_v3
round2
.............................round3
.......................................................10x_cells_v2
round2
.............................round3
........................................................smarter_nuclei
round2
............................round3
...................................................10x_nuclei_v3_macosko
round2
.............................round3
.......................................................10x_nuclei_v3
round2
.............................round3
........................................................snatac_gene
round2
............................round3
.....................................................

## Normalize them

In [20]:
normalization_options = {
    'smarter_nuclei': 'TPM',
    'smarter_cells': 'TPM',
    'snatac_gene': 'TPM',
    '10x_nuclei_v3_macosko': 'CPM',
    '10x_cells_v3': 'CPM',
    '10x_nuclei_v3': 'CPM',
    '10x_cells_v2': 'CPM',
#     'snmcseq_gene': 'MC',
}

In [23]:
for mod in mods_updated:
    print(mod)
    option = normalization_options[mod]
    for _round in rounds:
        print(_round)
        f = '/cndd2/fangming/miniatlas/cluster_level/gene_counts_{}_{}.tsv'.format(mod, _round)
        fout = '/cndd2/fangming/miniatlas/cluster_level/gene_features_{}_{}.tsv'.format(mod, _round)
        gene_counts = pd.read_csv(f, sep='\t', index_col=0)
        gene_features_new = pd.DataFrame(index=gene_counts.index, columns=gene_counts.columns)
        print(gene_counts.shape)
        gene_counts = gene_counts.dropna(axis=1)
        print(gene_counts.shape)
        
        if option == 'CPM':
            gene_features = snmcseq_utils.logcpm(gene_counts)
        elif option == 'TPM':
            gene_features = snmcseq_utils.logtpm(gene_counts, gene_lengths.reindex(gene_counts.index))
        print(gene_features.shape)
        gene_features_new.update(gene_features)
        print(gene_features_new.shape)
        gene_features_new.to_csv(fout, sep='\t', na_rep='NA', header=True, index=True)
        
#         break
#     break

smarter_cells
round2
(32324, 29)
(32324, 29)
(32324, 29)
(32324, 29)
round3
(32324, 56)
(32324, 54)
(32324, 54)
(32324, 56)
10x_cells_v3
round2
(31053, 29)
(31053, 29)
(31053, 29)
(31053, 29)
round3
(31053, 56)
(31053, 55)
(31053, 55)
(31053, 56)
10x_cells_v2
round2
(31053, 29)
(31053, 29)
(31053, 29)
(31053, 29)
round3
(31053, 56)
(31053, 56)
(31053, 56)
(31053, 56)
smarter_nuclei
round2
(32324, 29)
(32324, 28)
(32324, 28)
(32324, 29)
round3
(32324, 56)
(32324, 51)
(32324, 51)
(32324, 56)
10x_nuclei_v3_macosko
round2
(24809, 29)
(24809, 29)
(24809, 29)
(24809, 29)
round3
(24809, 56)
(24809, 55)
(24809, 55)
(24809, 56)
10x_nuclei_v3
round2
(31053, 29)
(31053, 29)
(31053, 29)
(31053, 29)
round3
(31053, 56)
(31053, 56)
(31053, 56)
(31053, 56)
snatac_gene
round2
(53278, 29)
(53278, 28)
(53278, 28)
(53278, 29)
round3
(53278, 56)
(53278, 53)
(53278, 53)
(53278, 56)
